In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [3]:
import helper
data = helper.load_RWHAR()

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 8)

train, val = helper.split(dtset, val_pc = 0.25)
train_weight = helper.dist(train, 8)
val_weight = helper.dist(val, 8)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True, num_workers = 10, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 32, num_workers = 10, pin_memory = True)

File exists. Loading
Selecting location :  forearm
Windowing
Done!
[[0.0984083 ]
 [0.09577236]
 [0.02284478]
 [0.15589199]
 [0.16407016]
 [0.15545267]
 [0.15038356]
 [0.15717617]] [[0.09773902]
 [0.0933793 ]
 [0.02250836]
 [0.14823076]
 [0.17053635]
 [0.15421271]
 [0.15542938]
 [0.15796411]]


In [7]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(in_channels = 6,input_size = 50, hidden_size = 512, output_size = 8, conv_filter = (3,5), conv_padding = (1,2))
model = Net(model = net, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 3, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_model_logs/', name = "RWHAR_model"),
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepConvNet      | 6 M   
1 | criterion | CrossEntropyLoss | 0     


1

In [8]:
from TransformerClassifier import TransformerClassifier
from Validation_model import Net
net = TransformerClassifier(in_channels = 6, output_size = 8, d_model = 50, nhead = 1, dim_feedforward = 2000, num_layers = 6)
model = Net(model = net, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 3, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'logs/', name = "RWHAR_transformer_model"),
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | TransformerClassifier | 1 M   
1 | criterion | CrossEntropyLoss      | 0     


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 5.93 GiB total capacity; 27.33 MiB already allocated; 5.50 MiB free; 56.00 MiB reserved in total by PyTorch)